In [1]:
%run base.ipynb

jit after {'jit': True, 'jit_options': {'flags': '-Ofast'}, 'compiler': 'shell'}
number of joints = 4


In [2]:
# forward dynamics

x_next, rhs_xdd, states, u, dt, q_min, q_max ,arm_p, _ , _, u_clamped  = rig_dyn.forward_dynamics()

In [3]:
# forces and moments at the base
F_at_base = rig_dyn.get_base_forces()
F_at_base_eval = cs.Function(
    'F_at_base', [arm_ss.q, arm_ss.q_dot, arm_ss.q_ddot, arm_ss.sim_p, arm_ss.gravity, arm_ss.base_T, q_min, q_max], [cs.vertcat(F_at_base[3:6], F_at_base[0:3])])

F_minus_xdd = F_at_base_eval(arm_ss.q, arm_ss.q_dot, rhs_xdd, arm_ss.sim_p, arm_ss.base_gravity, arm_ss.base_T, q_min, q_max)

In [4]:
# # rl
# rl_F_next = cs.Function('Mnext', [states, u, arm_p, dt, q_min, q_max, arm_ss.gravity, arm_ss.noise], [x_next, u_clamped])
# rl_F_next.save("model.casadi")

In [5]:
F_next_ = cs.Function('Mnext', [states, u, arm_p, dt, q_min, q_max, arm_ss.gravity, arm_ss.base_gravity, arm_ss.base_T, arm_ss.noise], [x_next, F_minus_xdd])

In [6]:
q0 = [3.1, 0.7, 0.4, 2.1]
q0dot = [0.00751799, -0.0073626, 0.00281749, 0.0222385]
u0 = [0.0340454, -0.00441695, 0.0062739, -0.00505981]
state0 = cs.vertcat(q0, q0dot)
noise = [0.0340454, -0.00441695, 0.0062739, -0.00505981]
# noise = [0.0, 0.0, 0.0, 0.0]
nextState , f_eval_base  = F_next_(state0, u0, alpha.sim_p, alpha.delta_t, alpha.joint_min, alpha.joint_max, alpha.gravity, alpha.base_gravity,
                                    alpha.base_T0, noise)
print(nextState)

[3.1005, 0.699902, 0.400137, 2.09984, 0.0144958, -0.00263048, 0.00396732, -0.00508349]


In [7]:
f_eval_base

DM([0.00832758, 0.0184997, -10.4967, -0.0142212, 0.968102, 0.000263118])

In [8]:
q0 = [3.1, 0.7, 0.4, 2.1]
q0dot = [0.0, 0.0, 0.0, 0.0]
u0 = [0.0, 0.0, 0.0, 0.0]
state0 = cs.vertcat(q0, q0dot)

nextState , f_eval_base  = F_next_(state0, u0, alpha.sim_p, alpha.delta_t, alpha.joint_min, alpha.joint_max, alpha.gravity,
                                    alpha.base_gravity, 
                                    alpha.base_T0,
                                    noise
                                    )
print(nextState)

[3.10003, 0.69999, 0.400009, 2.1, 0.000877631, -0.000305773, 0.000276786, -2.36831e-05]


In [9]:
f_eval_base

DM([-0.00077408, 0.00599551, -10.5072, -0.015311, 0.967722, 0.000402794])

In [9]:
# # c , cpp or matlab code generation for forward dynamics
# F_next_.generate("F_next_.c")
# os.system(f"gcc -fPIC -shared F_next_.c -o libMnext.so")

0